In [503]:
import requests
import random
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime , timedelta
import re
import os



path = "//Users//hajiaga//Desktop//DataScience//chromedriver"

driver = webdriver.Chrome()

driver.implicitly_wait(5)


In [504]:
def azetoengdate(string):

    gun = string[:2]

    ay = str(re.findall(r'\D+', string)[0]).replace(" ","").lower()

    if ay == "yanvar":
        numay = "01"
    elif ay == "fevral":
        numay = "02"
    elif ay == "mart":
        numay = "03"
    elif ay == "aprel":
        numay = "04"
    elif ay == "may":
        numay = "05"
    elif ay == "iyun":
        numay = "06"
    elif ay == "iyul":
        numay = "07"
    elif ay == "avqust":
        numay = "08"
    elif ay == "sentyabr":
        numay = "09"
    elif ay == "oktyabr":
        numay = "10"
    elif ay == "noyabr":
        numay = "11"
    elif ay == "dekabr":
        numay = "12"
    else:
        numay ="Nan"
    
    return f"{numay} {gun}"


In [505]:
def scrape_new_window(href , dic):
    time.sleep(1)
    driver.get(href)

    dic["full_name"].append((driver.find_element(By.XPATH , "//h1[@class = 'product-title']")).text)

    temp = driver.find_element(By.XPATH , "//div[@class = 'product-properties__column']")

    dic["location"].append((temp.find_element(By.XPATH , "//span[@class = 'product-properties__i-value']")).text)

    comp_name = "other" if (temp.find_element(By.TAG_NAME , "a")).text == "Digər" else (temp.find_element(By.TAG_NAME , "a")).text
    
    dic["comp_name"].append(comp_name)

    tempp = driver.find_element(By.XPATH , "//div[@class = 'product-price__i product-price__i--bold']")
    
    temppspans = tempp.find_elements(By.TAG_NAME , "span")

    dic["price"].append(temppspans[0].text)

    dic["currency"].append(temppspans[1].text)

    temp1 = driver.find_element(By.XPATH , "//div[@class = 'product-info product-info__statistics']")

    temp2 = temp1.find_elements(By.TAG_NAME , "div")

    date = (str(temp2[1].find_element(By.TAG_NAME , "span").text)).split(',')[0]

    if date == "Bugün":
        today = datetime.today()

        dic["date"].append(today.strftime('%m %d'))
        
    elif date == "Dünən":

        yesterday = datetime.today() - timedelta(days=1)

        dic["date"].append(yesterday.strftime('%m %d'))

        
    else:
        dic["date"].append(azetoengdate(date))

    dic["description"].append(driver.find_element(By.XPATH , "//*[@id='js-lot-page']/div/main/section[3]/div/div/p").text)

    driver.back()

    return dic


In [506]:
path1 = "https:/tap.az"

driver.get(path1)

time.sleep(5)
try:
    category = driver.find_element(By.XPATH , "//a[@data-for = 'consumer-electronics']")

    driver.get(category.get_attribute('href'))

    dropdown = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'price']")

    dropdown.click()
    
    minprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_lower']")

    maxprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_upper']")

    minprice.send_keys("1350")

    maxprice.send_keys("2500")

    time.sleep(5)

    categories = driver.find_element(By.XPATH , "//div[@class = 'subcategories-inner']")

    uls = categories.find_elements(By.TAG_NAME , 'ul')

    for ul in uls:
        
        lis = ul.find_elements(By.TAG_NAME , 'li')

        for li in lis:

            if "noutbuk" in str(li.find_element(By.TAG_NAME , 'a').get_attribute('href')).lower():

                href = li.find_element(By.TAG_NAME , 'a').get_attribute('href')

                break
    
    driver.get(href)

    dropdown_new = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'new']")

    dropdown_new.click()

    dropdown_new.find_element(By.TAG_NAME,"ul").find_elements(By.TAG_NAME , 'li')[1].click()
    
    time.sleep(3)

    dic_comp = {}

    dic_comp["location"] = []

    dic_comp["date"] = []

    dic_comp["link"] = []

    dic_comp["price"] = []

    dic_comp["comp_name"] = []

    dic_comp["full_name"] = []

    dic_comp["currency"] = []

    dic_comp["description"] = []

    last_length = 0

    count_= str(href).count("/")+1

    while True:

        products = driver.find_elements(By.CSS_SELECTOR, f"#content > div > div > div.categories-products.js-categories-products > div.js-endless-container.products.endless-products > div")

        if len(products) == last_length:
            break
       
        i = 0

        for product in products:
            
            i+=1

            if i>last_length:
                
                if ("pagination" not in product.get_attribute("class")):
                   
                    a = product.find_element(By.TAG_NAME , "a")

                    if (str(a.get_attribute("href")).count('/') == count_):

                        dic_comp["link"].append(a.get_attribute("href"))

                        dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)
    
        last_length = len(products)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(10)





    # The final problem is that i count until 48 but after that website is getting refreshed so i need to make a function for that and stop it gettinf the same thing after 48
            

    #Ilk usula don her defe scroll ele yeniden tap butun seyleri biraz vaxt alacaq
        


    
        
finally:
    driver.close()



KeyboardInterrupt: 

In [507]:
print(dic_comp)

{'location': ['Bakı', 'Bakı', 'Bakı', 'Bakı', 'Bakı', 'Bakı'], 'date': ['04 02', '04 02', '04 02', '04 02', '04 02', '04 02'], 'link': ['https://tap.az/elanlar/elektronika/noutbuklar/40827683', 'https://tap.az/elanlar/elektronika/noutbuklar/40618726', 'https://tap.az/elanlar/elektronika/noutbuklar/40953195', 'https://tap.az/elanlar/elektronika/noutbuklar/40982563', 'https://tap.az/elanlar/elektronika/noutbuklar/41025283', 'https://tap.az/elanlar/elektronika/noutbuklar/40021517', 'https://tap.az/elanlar/elektronika/noutbuklar/41081024'], 'price': ['1 420', '1 399', '2 222', '1 699', '1 649', '1 500'], 'comp_name': ['HP', 'Asus', 'Asus', 'Asus', 'Apple Macbook', 'Dell'], 'full_name': ['Noutbuk "HP Spectre x360"', 'Asus Tuf Gaming', 'Asus Tuf Gaming', 'Asus Tuf Gaming F15', 'Apple Macbook Air M2', 'Noutbuk "Dell G15 5511"'], 'currency': ['AZN', 'AZN', 'AZN', 'AZN', 'AZN', 'AZN'], 'description': ['Hp Spectre x360 2022/11, ikinci əl\n14-ef0013dx', '__𝐙𝐞𝐦𝐚𝐧𝐞𝐭𝐥𝐢 𝐍𝐨𝐭𝐞𝐛𝐨𝐨𝐤𝐥𝐚𝐫𝐢𝐧 𝐓𝐞𝐤 𝐔𝐧𝐯𝐚𝐧𝐢__\n__

for i in dic_comp.keys():
    print(len(dic_comp[i]))
dftapaz = pd.DataFrame.from_dict(dic_comp)

filename = "Users/hajiaga/Desktop/DataScience/tapazdata.csv"
filename2 = "Users/hajiaga/Desktop/DataScience/tapazdata.xlsx"
if os.path.exists(filename):
    os.remove(filename)
if os.path.exists(filename2):
    os.remove(filename2)
dftapaz.to_csv(filename, index=False) 
dftapaz.to_excel(filename2, index=False) 

dftapaz